In [6]:
# %%
from __future__ import annotations

from pathlib import Path
from typing import Optional, List, Dict, Any
import re

import pandas as pd
import plotly.graph_objects as go

# =========================
# Config
# =========================
PER_IMAGE_DIR = Path("./_eval_exports_per_images") / "sahi_null_v2"
FO_BASE = "https://fiftyone.tianqiyao.men"

# image-level 你想在点图里看的列
IMAGE_Y_COLS = ["tp_img", "fp_img", "fn_img", "pred_count_img", "hit_img"]

# daily-level 聚合频率：按天 "D"；按小时用 "H"
DAILY_FREQ = "H"

# daily-level 是否做异常高亮（基于 pred_count 的 z-score）
ENABLE_ANOMALY_HIGHLIGHT = True
ANOMALY_Z = 2.5  # 越小越敏感

# 输出目录（每个csv旁边也可以，这里统一放在csv所在目录下 _plots）
PLOTS_DIRNAME = "_plots_prod"


# =========================
# Utils
# =========================
def find_per_image_csvs(root: Path) -> List[Path]:
    return sorted(root.rglob("image_level_*.csv"))

def make_fo_url(dataset_name: str, sample_id: str, fo_base: str = FO_BASE) -> str:
    return f"{fo_base}/datasets/{dataset_name}?id={sample_id}"

def _safe_stem(s: str) -> str:
    s = re.sub(r"[^a-zA-Z0-9._-]+", "_", s)
    return s[:220]

def _build_hovertemplate(y_col: str, hover_cols: List[str]) -> str:
    lines = [f"<b>{y_col}</b>: %{{y}}<br>"]
    for i, c in enumerate(hover_cols, start=1):
        lines.append(f"{c}: %{{customdata[{i}]}}<br>")
    lines.append("<b>Click</b> point to open FiftyOne<br>")
    return "".join(lines)

def _ensure_datetime_x(df: pd.DataFrame) -> str:
    """
    Prefer capture_datetime if parseable; else fallback to index.
    Returns x_col name.
    """
    x_col = None
    if "capture_datetime" in df.columns:
        df["capture_datetime"] = pd.to_datetime(df["capture_datetime"], errors="coerce")
        if df["capture_datetime"].notna().any():
            x_col = "capture_datetime"

    if x_col is None:
        df["_index"] = range(len(df))
        x_col = "_index"

    df.sort_values(x_col, inplace=True)
    return x_col

def _add_url_column(df: pd.DataFrame) -> None:
    df["sample_id"] = df["sample_id"].astype(str)
    df["url"] = df.apply(lambda r: make_fo_url(r["dataset_name"], r["sample_id"], FO_BASE), axis=1)

def _choose_hover_cols(df: pd.DataFrame) -> List[str]:
    cols = []
    for c in [
        "filepath",
        "gt_count_img",
        "pred_count_img",
        "tp_img",
        "fp_img",
        "fn_img",
        "hit_img",
        "confidence_threshold",
        "iou_threshold",
        "model_tag",
        "mean_confidence",   # 如果你有这列就会自动带上
        "avg_confidence",
    ]:
        if c in df.columns:
            cols.append(c)
    return cols


# =========================
# Layer B: Image-level HTML (click -> FiftyOne)
# =========================
def draw_image_level_html(
    df: pd.DataFrame,
    out_html: Path,
    y_cols: List[str],
    title: str,
    show_plot: bool = False,
) -> None:
    required_cols = {"dataset_name", "sample_id", "filepath"}
    missing = required_cols - set(df.columns)
    if missing:
        print(f"[SKIP] missing columns {missing} -> {out_html.name}")
        return

    available_y = [c for c in y_cols if c in df.columns]
    if not available_y:
        print(f"[SKIP] no y_cols available -> {out_html.name}")
        return

    x_col = _ensure_datetime_x(df)
    _add_url_column(df)
    hover_cols = _choose_hover_cols(df)

    fig = go.Figure()

    # 多指标散点（每个指标一条trace）
    for y in available_y:
        yv = pd.to_numeric(df[y], errors="coerce").fillna(0)
        fig.add_trace(
            go.Scatter(
                x=df[x_col],
                y=yv,
                mode="markers",
                name=y,
                marker=dict(size=5, opacity=0.75),
                customdata=df[["url"] + hover_cols].values,
                hovertemplate=_build_hovertemplate(y_col=y, hover_cols=hover_cols),
            )
        )

    fig.update_layout(
        title=title,
        xaxis=dict(
            title="Time" if x_col == "capture_datetime" else "Index",
            rangeslider=dict(visible=False),  # image-level 一般不需要 rangeslider（更干净）
        ),
        yaxis=dict(title="Value"),
        template="plotly_white",
        margin=dict(l=70, r=30, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    # 点击打开FiftyOne
    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plot = document.getElementsByClassName('js-plotly-plot')[0];
        if (!plot) { return; }

        plot.on('plotly_click', function(data) {
            var url = data?.points?.[0]?.customdata?.[0];
            if (url) window.open(url, '_blank');
        });
    });
    """

    out_html.parent.mkdir(parents=True, exist_ok=True)
    fig.write_html(out_html, include_plotlyjs="cdn", full_html=True, post_script=post_script)
    print(f"[SAVE][image] {out_html}")

    if show_plot:
        fig.show()


# =========================
# Layer A: Daily-level HTML (click day -> open that day's image-level HTML)
# =========================
def draw_daily_level_html_with_drilldown(
    df: pd.DataFrame,
    out_html: Path,
    per_day_html_relpaths: Dict[pd.Timestamp, str],  # day_start -> relative html path
    freq: str = "D",
    title: str = "Daily summary",
) -> None:
    if "capture_datetime" not in df.columns:
        print(f"[SKIP] no capture_datetime -> {out_html.name}")
        return

    df["capture_datetime"] = pd.to_datetime(df["capture_datetime"], errors="coerce")
    df = df.dropna(subset=["capture_datetime"]).copy()
    df = df.set_index("capture_datetime").sort_index()

    # daily aggregation
    agg_map = {}
    # 最核心：pred_count_img, hit_img
    if "pred_count_img" in df.columns:
        agg_map["pred_count_img"] = "sum"
    if "hit_img" in df.columns:
        agg_map["hit_img"] = "mean"

    # 如果你有tp/fp/fn，就聚合并算error
    for c in ["tp_img", "fp_img", "fn_img"]:
        if c in df.columns:
            agg_map[c] = "sum"

    # 如果你有平均置信度列，也可以带上
    conf_col = None
    for cand in ["mean_confidence", "avg_confidence"]:
        if cand in df.columns:
            conf_col = cand
            agg_map[cand] = "mean"
            break

    if not agg_map:
        print(f"[SKIP] no usable columns for daily -> {out_html.name}")
        return

    daily = df.resample(freq).agg(agg_map)

    if "fp_img" in daily.columns and "fn_img" in daily.columns:
        daily["error"] = daily["fp_img"].fillna(0) + daily["fn_img"].fillna(0)

    # anomaly detection (z-score on pred_count sum)
    anomaly_mask = None
    if ENABLE_ANOMALY_HIGHLIGHT and "pred_count_img" in daily.columns:
        s = daily["pred_count_img"].fillna(0)
        if s.std() > 1e-9:
            z = (s - s.mean()) / s.std()
            anomaly_mask = z.abs() >= ANOMALY_Z
        else:
            anomaly_mask = pd.Series(False, index=daily.index)

    # Prepare drilldown link per point (relative path)
    # We align by index timestamp (day_start)
    drill_links = []
    for ts in daily.index:
        drill_links.append(per_day_html_relpaths.get(ts, ""))

    fig = go.Figure()

    # 1) total detections (bar)
    if "pred_count_img" in daily.columns:
        fig.add_trace(
            go.Bar(
                x=daily.index,
                y=daily["pred_count_img"].fillna(0),
                name="Total detections (sum pred_count_img)",
                customdata=list(zip(drill_links)),
                hovertemplate="Date: %{x}<br>Total detections: %{y}<br><b>Click</b> to open image-level for this day<br><extra></extra>",
            )
        )

    # 2) hit rate (line, y2)
    if "hit_img" in daily.columns:
        fig.add_trace(
            go.Scatter(
                x=daily.index,
                y=daily["hit_img"],
                name="Hit rate (mean hit_img)",
                mode="lines+markers",
                yaxis="y2",
                hovertemplate="Date: %{x}<br>Hit rate: %{y:.3f}<br><extra></extra>",
            )
        )

    # 3) error (line) if exists
    if "error" in daily.columns:
        fig.add_trace(
            go.Scatter(
                x=daily.index,
                y=daily["error"].fillna(0),
                name="Error (sum fp+fn)",
                mode="lines+markers",
                hovertemplate="Date: %{x}<br>Error: %{y}<br><extra></extra>",
            )
        )

    # 4) avg confidence (line) if exists
    if conf_col and conf_col in daily.columns:
        fig.add_trace(
            go.Scatter(
                x=daily.index,
                y=daily[conf_col],
                name=f"Avg confidence ({conf_col})",
                mode="lines+markers",
                yaxis="y2",
                hovertemplate="Date: %{x}<br>Avg conf: %{y:.3f}<br><extra></extra>",
            )
        )

    # Highlight anomaly windows by adding translucent rectangles
    shapes = []
    if anomaly_mask is not None and anomaly_mask.any():
        # each anomalous bin: [ts, ts+freq)
        # for D: +1 day; for H: +1 hour
        offset = pd.Timedelta(days=1) if freq.upper() == "D" else pd.Timedelta(hours=1)
        for ts, is_anom in anomaly_mask.items():
            if not bool(is_anom):
                continue
            shapes.append(
                dict(
                    type="rect",
                    xref="x",
                    yref="paper",
                    x0=ts,
                    x1=ts + offset,
                    y0=0,
                    y1=1,
                    fillcolor="red",
                    opacity=0.10,
                    line_width=0,
                    layer="below",
                )
            )

    fig.update_layout(
        title=title,
        xaxis=dict(title="Date"),
        yaxis=dict(title="Counts"),
        yaxis2=dict(
            title="Rate / Confidence",
            overlaying="y",
            side="right",
            range=[0, 1],
        ),
        template="plotly_white",
        margin=dict(l=70, r=70, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        shapes=shapes,
    )

    # click a bar -> open per-day image HTML (relative path)
    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plot = document.getElementsByClassName('js-plotly-plot')[0];
        if (!plot) return;

        plot.on('plotly_click', function(data) {
            var rel = data?.points?.[0]?.customdata?.[0];
            if (rel) window.open(rel, '_blank');
        });
    });
    """

    out_html.parent.mkdir(parents=True, exist_ok=True)
    fig.write_html(out_html, include_plotlyjs="cdn", full_html=True, post_script=post_script)
    print(f"[SAVE][daily]  {out_html}")


# =========================
# Orchestrator: for each per-image CSV, generate daily + per-day image drilldowns
# =========================
def process_one_csv(
    csv_path: Path,
    y_cols: List[str],
    freq: str = "D",
) -> None:
    df = pd.read_csv(csv_path)

    # Basic required cols for image-level
    required = {"dataset_name", "sample_id", "filepath"}
    if not required.issubset(df.columns):
        print(f"[SKIP] required cols missing in {csv_path}")
        return

    # Need capture_datetime for daily layer; if missing, we still generate a single image-level file
    has_dt = "capture_datetime" in df.columns

    plot_root = csv_path.parent / PLOTS_DIRNAME / _safe_stem(csv_path.stem)
    plot_root.mkdir(parents=True, exist_ok=True)

    # Always generate full image-level (all images)
    full_img_html = plot_root / f"{_safe_stem(csv_path.stem)}__image_all.html"
    draw_image_level_html(
        df=df.copy(),
        out_html=full_img_html,
        y_cols=y_cols,
        title=f"{csv_path.stem} | IMAGE (all)",
        show_plot=False,
    )

    if not has_dt:
        print(f"[NOTE] no capture_datetime -> skip daily layer for {csv_path.name}")
        return

    # Parse dt and build per-day splits
    df["capture_datetime"] = pd.to_datetime(df["capture_datetime"], errors="coerce")
    df2 = df.dropna(subset=["capture_datetime"]).copy()
    if df2.empty:
        print(f"[SKIP] capture_datetime all NaT -> {csv_path.name}")
        return

    # Define bin start per freq
    # For D: day start; For H: hour start
    df2["_bin"] = df2["capture_datetime"].dt.floor("d" if freq.lower() == "d" else "h")

    per_day_html_relpaths: Dict[pd.Timestamp, str] = {}

    # Generate per-bin image-level HTML (drilldown targets)
    for bin_ts, g in df2.groupby("_bin"):
        bin_ts = pd.Timestamp(bin_ts)

        day_tag = bin_ts.strftime("%Y%m%d") if freq.lower() == "d" else bin_ts.strftime("%Y%m%d_%H")
        out_day_html = plot_root / f"{_safe_stem(csv_path.stem)}__image_{freq}_{day_tag}.html"

        draw_image_level_html(
            df=g.drop(columns=["_bin"]).copy(),
            out_html=out_day_html,
            y_cols=y_cols,
            title=f"{csv_path.stem} | IMAGE ({freq}={day_tag})",
            show_plot=False,
        )

        # Store relative path from daily html to per-day html
        per_day_html_relpaths[bin_ts] = out_day_html.name  # same folder, so name is enough

    # Generate daily summary HTML with drilldown
    daily_html = plot_root / f"{_safe_stem(csv_path.stem)}__daily_{freq}.html"
    draw_daily_level_html_with_drilldown(
        df=df.copy(),
        out_html=daily_html,
        per_day_html_relpaths=per_day_html_relpaths,
        freq=freq,
        title=f"{csv_path.stem} | DAILY ({freq}) summary (click bar to drill down)",
    )

    # Optional: a tiny index file for convenience
    index_html = plot_root / "INDEX.txt"
    index_html.write_text(
        "\n".join([
            f"CSV: {csv_path}",
            f"Daily summary: {daily_html.name}",
            f"Image all: {full_img_html.name}",
            f"Per-{freq} images: {len(per_day_html_relpaths)} html files",
        ]),
        encoding="utf-8",
    )


def batch_process(
    root: Path,
    y_cols: List[str],
    freq: str = "D",
    limit: Optional[int] = None,
) -> None:
    csvs = find_per_image_csvs(root)
    print("[INFO] Found CSVs:", len(csvs))
    if limit is not None:
        csvs = csvs[:limit]
        print("[INFO] Using first:", len(csvs))

    for p in csvs:
        print("\n" + "=" * 80)
        print("[CSV]", p)
        process_one_csv(p, y_cols=y_cols, freq=freq)

    print("\nDone.")


# =========================
# Run
# =========================
batch_process(
    root=PER_IMAGE_DIR,
    y_cols=IMAGE_Y_COLS,
    freq=DAILY_FREQ,
    limit=10,   # 先跑前10个验证；确认没问题再去掉limit
)


[INFO] Found CSVs: 126
[INFO] Using first: 10

[CSV] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50.csv
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50__image_all.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50__image_H_20240606_06.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50__image_H_2

/tmp/ipykernel_1243479/1034236670.py:212: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50__image_H_20240606_14.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50__image_H_20240606_20.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50__image_H_20240607_07.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image

/tmp/ipykernel_1243479/1034236670.py:212: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50__image_H_20240606_14.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50__image_H_20240606_20.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50__image_H_20240607_07.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50/image

...

/tmp/ipykernel_1243479/1034236670.py:212: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50__image_H_20240606_06.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50__image_H_20240606_07.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50__image_H_20240606_08.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50/image

/tmp/ipykernel_1243479/1034236670.py:212: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a04_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_a04_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50__image_H_20240606_06.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a04_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_a04_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50__image_H_20240606_07.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a04_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_a04_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50__image_H_20240606_08.html
[SAVE][image] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots_prod/image_level_a04_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image

/tmp/ipykernel_1243479/1034236670.py:212: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [18]:
from __future__ import annotations

import csv
from pathlib import Path
from typing import List, Optional, Dict
import re

import pandas as pd
import plotly.graph_objects as go


# =========================
# Config
# =========================
PER_IMAGE_DIR = Path("./_eval_exports_per_images") / "sahi_null_v2"
FO_BASE = "https://fiftyone.tianqiyao.men"

# 输出根目录：会在里面生成 daily_overview.html + 每天一个 image_level_YYYY-MM-DD.html
OUT_ROOT = PER_IMAGE_DIR / "_prod_kiss_reports"

# 一次处理多少个 CSV（你可以先 1~3 测试）
LIMIT: Optional[int] = 10

# image-level 页面：y 轴选什么（KISS：用 pred_count_img）
IMAGE_Y_COL = "pred_count_img"

# =========================
# Utils
# =========================
def find_per_image_csvs(root: Path) -> List[Path]:
    return sorted(root.rglob("image_level_*.csv"))


def safe_name(s: str) -> str:
    s = re.sub(r"[^a-zA-Z0-9._-]+", "_", s)
    return s[:200]


def make_fo_url(dataset_name: str, sample_id: str, fo_base: str = FO_BASE) -> str:
    return f"{fo_base}/datasets/{dataset_name}?id={sample_id}"


def ensure_time_cols(df: pd.DataFrame) -> pd.DataFrame:
    # capture_datetime -> datetime
    if "capture_datetime" in df.columns:
        df["capture_datetime"] = pd.to_datetime(df["capture_datetime"], errors="coerce")

    # capture_date: 如果没有就从 capture_datetime 派生；如果有也统一成 string YYYY-MM-DD
    if "capture_date" not in df.columns or df["capture_date"].isna().all():
        if "capture_datetime" in df.columns:
            df["capture_date"] = df["capture_datetime"].dt.date.astype(str)
        else:
            # 兜底：全放一个日期
            df["capture_date"] = "unknown_date"
    else:
        # 统一格式
        df["capture_date"] = df["capture_date"].astype(str)

    return df


def ensure_event_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    保证有 hit_img/miss_img/false_alarm_img/correct_reject_img 四分类（每张图 0/1）
    优先用你已有列；没有就用 gt_count_img/pred_count_img 推导；
    如果连 gt_count_img 都没有，就退化为 production 模式：hit_img = pred>0, 其余置 0
    """
    # 统一数值列
    for c in ["gt_count_img", "pred_count_img", "tp_img", "fp_img", "fn_img"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

    # 如果四分类都有，就确保是 int(0/1)
    cols = ["hit_img", "miss_img", "false_alarm_img", "correct_reject_img"]
    if all(c in df.columns for c in cols):
        for c in cols:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
        return df

    # 否则尝试推导
    has_gt = "gt_count_img" in df.columns
    has_pred = "pred_count_img" in df.columns

    if has_pred:
        pred_present = df["pred_count_img"] > 0
    else:
        pred_present = pd.Series(False, index=df.index)

    if has_gt:
        gt_present = df["gt_count_img"] > 0

        # 四分类（互斥完备）
        df["hit_img"] = (gt_present & pred_present).astype(int)
        df["miss_img"] = (gt_present & ~pred_present).astype(int)
        df["false_alarm_img"] = (~gt_present & pred_present).astype(int)
        df["correct_reject_img"] = (~gt_present & ~pred_present).astype(int)
    else:
        # 无 GT：只保留 hit_img（pred>0）
        df["hit_img"] = pred_present.astype(int)
        df["miss_img"] = 0
        df["false_alarm_img"] = 0
        df["correct_reject_img"] = 0

    return df


def add_fo_url(df: pd.DataFrame) -> pd.DataFrame:
    must = {"dataset_name", "sample_id"}
    if must.issubset(df.columns):
        df["sample_id"] = df["sample_id"].astype(str)
        df["url"] = df.apply(lambda r: make_fo_url(r["dataset_name"], r["sample_id"], FO_BASE), axis=1)
    else:
        df["url"] = ""
    return df


def pick_hover_cols(df: pd.DataFrame) -> List[str]:
    hover = []
    for c in [
        "filepath",
        "capture_datetime",
        "gt_count_img",
        "pred_count_img",
        "hit_img",
        "miss_img",
        "false_alarm_img",
        "correct_reject_img",
        "avg_confidence",
        "median_confidence",
        "confidence_threshold",
        "iou_threshold",
        "model_tag",
    ]:
        if c in df.columns:
            hover.append(c)
    return hover


def build_hovertemplate(y_col: str, hover_cols: List[str]) -> str:
    # customdata[0] is url; then hover_cols
    lines = [f"<b>{y_col}</b>: %{{y}}<br>"]
    for i, c in enumerate(hover_cols, start=1):
        lines.append(f"{c}: %{{customdata[{i}]}}<br>")
    lines.append("<b>Click</b> to open FiftyOne<br>")
    return "".join(lines)


# =========================
# Image-level page (per day)
# =========================
def write_image_level_day_html(
    df_day: pd.DataFrame,
    out_html: Path,
    title: str,
    y_col: str = "pred_count_img",
) -> None:
    df_day = df_day.copy()

    # x 轴：有时间就用时间；否则 index
    if "capture_datetime" in df_day.columns:
        df_day["capture_datetime"] = pd.to_datetime(df_day["capture_datetime"], errors="coerce")
        if df_day["capture_datetime"].notna().any():
            x = df_day["capture_datetime"]
            x_title = "capture_datetime"
        else:
            df_day["_index"] = range(len(df_day))
            x = df_day["_index"]
            x_title = "index"
    else:
        df_day["_index"] = range(len(df_day))
        x = df_day["_index"]
        x_title = "index"

    # y 轴：默认 pred_count_img；不存在则退化为 hit_img
    if y_col not in df_day.columns:
        y_col = "hit_img" if "hit_img" in df_day.columns else None
    if y_col is None:
        raise ValueError("No usable y_col for image-level plot")

    y = pd.to_numeric(df_day[y_col], errors="coerce").fillna(0)

    df_day = add_fo_url(df_day)
    hover_cols = pick_hover_cols(df_day)

    fig = go.Figure()

    # KISS: 一个散点就够了（想分颜色以后再加）
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="markers",
            name="images",
            marker=dict(size=6, opacity=0.8),
            customdata=df_day[["url"] + hover_cols].values,
            hovertemplate=build_hovertemplate(y_col=y_col, hover_cols=hover_cols),
        )
    )

    fig.update_layout(
        title=title,
        xaxis=dict(title=x_title, rangeslider=dict(visible=True)),
        yaxis=dict(title=y_col),
        template="plotly_white",
        margin=dict(l=70, r=30, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    # 点 image → 打开 FiftyOne
    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plot = document.getElementsByClassName('js-plotly-plot')[0];
        if (!plot) return;

        plot.on('plotly_click', function(e) {
            var url = e?.points?.[0]?.customdata?.[0];
            if (url) window.open(url, '_blank');
        });
    });
    """

    out_html.parent.mkdir(parents=True, exist_ok=True)
    fig.write_html(out_html, include_plotlyjs="cdn", full_html=True, post_script=post_script)
    print(f"[SAVE][image-day] {out_html}")


# =========================
# Daily overview page (stacked bar + 2 lines) with drilldown
# =========================
def write_daily_overview_html(
    df: pd.DataFrame,
    out_html: Path,
    day_to_file: Dict[str, str],  # capture_date -> html filename (relative)
    title: str,
) -> None:
    # 聚合：四分类堆叠（image-level）
    daily = (
        df.groupby("capture_date", dropna=False)
          .agg(
              images=("sample_id", "count") if "sample_id" in df.columns else ("filepath", "count"),
              hit=("hit_img", "sum"),
              miss=("miss_img", "sum"),
              false_alarm=("false_alarm_img", "sum"),
              correct_reject=("correct_reject_img", "sum"),
          )
          .reset_index()
    )

    # 排序：按日期字符串排序即可（YYYY-MM-DD）
    daily = daily.sort_values("capture_date")

    # 两条线：hit_rate / error_rate（eval 有 GT 才有意义，但 miss/false_alarm 没有也会是 0）
    daily["hit_rate"] = daily["hit"] / daily["images"].clip(lower=1)
    daily["error_rate"] = (daily["miss"] + daily["false_alarm"]) / daily["images"].clip(lower=1)

    # drilldown link：每一天一个 html
    daily["drill_html"] = daily["capture_date"].map(day_to_file).fillna("")

    x = daily["capture_date"]

    fig = go.Figure()

    # stacked bars: 4 categories (image-level)
    # 每条 trace customdata 放 “当日 drilldown html”
    for col, name in [
        ("hit", "Hit (GT=1 & Pred=1)"),
        ("miss", "Miss (GT=1 & Pred=0)"),
        ("false_alarm", "False Alarm (GT=0 & Pred=1)"),
        ("correct_reject", "Correct Reject (GT=0 & Pred=0)"),
    ]:
        fig.add_trace(
            go.Bar(
                x=x,
                y=daily[col],
                name=name,
                customdata=daily[["drill_html"]].values,
                hovertemplate="Date: %{x}<br>Count: %{y}<br><b>Click</b> to drill down<br><extra></extra>",
            )
        )

    # two lines on y2 (KISS)
    fig.add_trace(
        go.Scatter(
            x=x,
            y=daily["hit_rate"],
            name="Hit rate",
            mode="lines+markers",
            yaxis="y2",
            hovertemplate="Date: %{x}<br>Hit rate: %{y:.3f}<br><extra></extra>",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=x,
            y=daily["error_rate"],
            name="Error rate (miss+false_alarm)/images",
            mode="lines+markers",
            yaxis="y2",
            hovertemplate="Date: %{x}<br>Error rate: %{y:.3f}<br><extra></extra>",
        )
    )

    fig.update_layout(
        title=title,
        barmode="stack",
        xaxis=dict(title="capture_date", rangeslider=dict(visible=True)),
        yaxis=dict(title="Images (stacked by outcome)"),
        yaxis2=dict(title="Rate", overlaying="y", side="right", range=[0, 1]),
        template="plotly_white",
        margin=dict(l=70, r=70, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    # 点击任何 bar → 打开该日的 image-level html
    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plot = document.getElementsByClassName('js-plotly-plot')[0];
        if (!plot) return;

        plot.on('plotly_click', function(e) {
            var rel = e?.points?.[0]?.customdata?.[0];
            if (rel) window.open(rel, '_blank');
        });
    });
    """

    out_html.parent.mkdir(parents=True, exist_ok=True)
    fig.write_html(out_html, include_plotlyjs="cdn", full_html=True, post_script=post_script)
    print(f"[SAVE][daily] {out_html}")


# =========================
# Main: one CSV -> daily + per-day image-level
# =========================
def process_one_csv(csv_path: Path) -> None:
    df = pd.read_csv(csv_path)

    # 必需字段检查（最少要能定位 image）
    if not {"filepath"}.issubset(df.columns):
        print(f"[SKIP] missing filepath: {csv_path}")
        return

    # 时间 + 四分类
    df = ensure_time_cols(df)
    df = ensure_event_cols(df)

    # 输出目录：按这个 csv stem 分开
    report_dir = OUT_ROOT / csv_path.parent.name / safe_name(csv_path.stem)
    report_dir.mkdir(parents=True, exist_ok=True)

    # 生成每一天的 image-level html，并记录 drilldown 文件名
    day_to_file: Dict[str, str] = {}

    for day, g in df.groupby("capture_date"):
        day = str(day)
        day_file = f"image_level_{day}.html"
        out_day_html = report_dir / day_file

        write_image_level_day_html(
            df_day=g,
            out_html=out_day_html,
            title=f"{csv_path.stem} | Image-level | {day}",
            y_col=IMAGE_Y_COL,
        )
        day_to_file[day] = day_file  # relative path (same folder)

    # 生成 daily overview html（点击 → 打开对应 day html）
    out_daily_html = report_dir / "daily_overview.html"
    write_daily_overview_html(
        df=df,
        out_html=out_daily_html,
        day_to_file=day_to_file,
        title=f"{csv_path.stem} | Daily overview (click bars to drill down)",
    )

    # 写一个简单入口说明
    (report_dir / "INDEX.txt").write_text(
        "\n".join([
            f"CSV: {csv_path}",
            f"Daily: {out_daily_html.name}",
            f"Days: {len(day_to_file)}",
            "Click bars in daily_overview.html to open the corresponding image-level page.",
            "Click points in image-level page to open FiftyOne sample URL.",
        ]),
        encoding="utf-8"
    )

    print(f"[DONE] report_dir = {report_dir}")


def main():
    csvs = find_per_image_csvs(PER_IMAGE_DIR)
    print("[INFO] Found CSVs:", len(csvs))
    if LIMIT is not None:
        csvs = csvs[:LIMIT]
        print("[INFO] Using first:", len(csvs))

    for p in csvs:
        print("\n" + "=" * 90)
        print("[CSV]", p)
        process_one_csv(p)

    print("\nAll done. Open the generated daily_overview.html files under:")
    print(OUT_ROOT)


if __name__ == "__main__":
    main()


[INFO] Found CSVs: 504
[INFO] Using first: 10

[CSV] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50.csv
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-06.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-07.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-08.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0

### 3

In [21]:
from __future__ import annotations

from pathlib import Path
from typing import List, Optional, Dict
import re
import html

import pandas as pd
import plotly.graph_objects as go


# =========================
# Config
# =========================
PER_IMAGE_DIR = Path("./_eval_exports_per_images") / "sahi_null_v2"
FO_BASE = "https://fiftyone.tianqiyao.men"

OUT_ROOT = PER_IMAGE_DIR / "_prod_kiss_reports"

LIMIT: Optional[int] = None  # set None to run all

TOP_N = 20


# =========================
# Utils
# =========================
def find_per_image_csvs(root: Path) -> List[Path]:
    return sorted(root.rglob("image_level_*.csv"))


def safe_name(s: str) -> str:
    s = re.sub(r"[^a-zA-Z0-9._-]+", "_", s)
    return s[:200]


def make_fo_url(dataset_name: str, sample_id: str, fo_base: str = FO_BASE) -> str:
    return f"{fo_base}/datasets/{dataset_name}?id={sample_id}"


def ensure_time_cols(df: pd.DataFrame) -> pd.DataFrame:
    if "capture_datetime" in df.columns:
        df["capture_datetime"] = pd.to_datetime(df["capture_datetime"], errors="coerce")

    if "capture_date" not in df.columns or df["capture_date"].isna().all():
        if "capture_datetime" in df.columns:
            df["capture_date"] = df["capture_datetime"].dt.date.astype(str)
        else:
            df["capture_date"] = "unknown_date"
    else:
        df["capture_date"] = df["capture_date"].astype(str)

    return df


def ensure_event_cols(df: pd.DataFrame) -> pd.DataFrame:
    # numeric normalize
    for c in ["gt_count_img", "pred_count_img", "tp_img", "fp_img", "fn_img"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

    # Recompute mutually-exclusive image-level outcomes (KISS + robust)
    has_gt = "gt_count_img" in df.columns
    has_pred = "pred_count_img" in df.columns
    pred_present = (df["pred_count_img"] > 0) if has_pred else pd.Series(False, index=df.index)

    if has_gt:
        gt_present = df["gt_count_img"] > 0
        df["hit_img"] = (gt_present & pred_present).astype(int)
        df["miss_img"] = (gt_present & ~pred_present).astype(int)
        df["false_alarm_img"] = (~gt_present & pred_present).astype(int)
        df["correct_reject_img"] = (~gt_present & ~pred_present).astype(int)
    else:
        # production fallback (no GT): hit vs no-hit
        df["hit_img"] = pred_present.astype(int)
        df["miss_img"] = 0
        df["false_alarm_img"] = 0
        df["correct_reject_img"] = 0

    return df


def add_fo_url(df: pd.DataFrame) -> pd.DataFrame:
    must = {"dataset_name", "sample_id"}
    if must.issubset(df.columns):
        df["sample_id"] = df["sample_id"].astype(str)
        df["url"] = df.apply(lambda r: make_fo_url(r["dataset_name"], r["sample_id"], FO_BASE), axis=1)
    else:
        df["url"] = ""
    return df


def infer_event_label(df: pd.DataFrame) -> pd.Series:
    """
    event label used for coloring
    - With GT: Hit/Miss/False Alarm/Correct Reject (mutually exclusive)
    - No GT: Hit/No-Hit
    """
    has_gt = "gt_count_img" in df.columns

    if has_gt:
        cond_hit = df.get("hit_img", 0).astype(int) == 1
        cond_miss = df.get("miss_img", 0).astype(int) == 1
        cond_fa = df.get("false_alarm_img", 0).astype(int) == 1
        cond_cr = df.get("correct_reject_img", 0).astype(int) == 1

        out = pd.Series(["Unknown"] * len(df), index=df.index)
        out[cond_hit] = "Hit"
        out[cond_miss] = "Miss"
        out[cond_fa] = "False Alarm"
        out[cond_cr] = "Correct Reject"
        return out

    pred_present = (pd.to_numeric(df.get("pred_count_img", 0), errors="coerce").fillna(0) > 0)
    return pd.Series(["Hit" if b else "No-Hit" for b in pred_present], index=df.index)


def _x_for_day(df_day: pd.DataFrame):
    """Return (x_series, x_title, df_day_with_index_if_needed)."""
    df_day = df_day.copy()
    if "capture_datetime" in df_day.columns:
        df_day["capture_datetime"] = pd.to_datetime(df_day["capture_datetime"], errors="coerce")
        if df_day["capture_datetime"].notna().any():
            return df_day["capture_datetime"], "capture_datetime", df_day
    df_day["_index"] = range(len(df_day))
    return df_day["_index"], "index", df_day


def _make_hover_cols(df: pd.DataFrame) -> List[str]:
    cols = []
    for c in [
        "filepath",
        "capture_datetime",
        "gt_count_img",
        "pred_count_img",
        "tp_img",
        "fp_img",
        "fn_img",
        "avg_confidence",
        "median_confidence",
        "confidence_threshold",
        "iou_threshold",
        "model_tag",
        "event",
        "err_obj",
        "tp_ratio",  # added
    ]:
        if c in df.columns:
            cols.append(c)
    return cols


def _build_hovertemplate(y_label: str, hover_cols: List[str]) -> str:
    lines = [f"<b>{y_label}</b>: %{{y}}<br>"]
    for i, c in enumerate(hover_cols, start=1):
        lines.append(f"{c}: %{{customdata[{i}]}}<br>")
    lines.append("<b>Click</b> to open FiftyOne<br>")
    return "".join(lines)


def make_top_table_html(df_day: pd.DataFrame, top_n: int = 20) -> str:
    """
    Top-N table: sort by fp+fn (err_obj) desc, then by fn desc, then fp desc.
    """
    d = df_day.copy()
    d = add_fo_url(d)

    for c in ["pred_count_img", "gt_count_img", "avg_confidence", "tp_img", "fp_img", "fn_img", "err_obj", "tp_ratio"]:
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors="coerce")

    d["event"] = infer_event_label(d)
    d["err_obj"] = pd.to_numeric(d.get("fp_img", 0), errors="coerce").fillna(0) + pd.to_numeric(d.get("fn_img", 0), errors="coerce").fillna(0)

    d["_err"] = d["err_obj"].fillna(0)
    d["_fn"] = pd.to_numeric(d.get("fn_img", 0), errors="coerce").fillna(0)
    d["_fp"] = pd.to_numeric(d.get("fp_img", 0), errors="coerce").fillna(0)

    d = d.sort_values(["_err", "_fn", "_fp"], ascending=[False, False, False]).head(top_n)

    def esc(x: object) -> str:
        return html.escape("" if pd.isna(x) else str(x))

    def short_path(p: str, maxlen: int = 70) -> str:
        p = "" if pd.isna(p) else str(p)
        if len(p) <= maxlen:
            return p
        return "…" + p[-(maxlen - 1):]

    headers = [
        "Event", "capture_datetime", "filepath",
        "gt", "pred", "tp", "fp", "fn", "fp+fn",
        "tp/gt", "avg_conf", "Link"
    ]

    rows = []
    for _, r in d.iterrows():
        url = r.get("url", "")
        link = f'<a href="{esc(url)}" target="_blank">Open</a>' if url else ""
        fp = short_path(r.get("filepath", ""), 70)

        cap = r.get("capture_datetime", "")
        cap = "" if pd.isna(cap) else str(cap)

        tp_ratio = r.get("tp_ratio", "")
        tp_ratio = "" if pd.isna(tp_ratio) else f"{float(tp_ratio):.3f}"

        rows.append([
            esc(r.get("event", "")),
            esc(cap),
            esc(fp),
            esc(r.get("gt_count_img", "")) if "gt_count_img" in d.columns else "",
            esc(r.get("pred_count_img", "")) if "pred_count_img" in d.columns else "",
            esc(r.get("tp_img", "")) if "tp_img" in d.columns else "",
            esc(r.get("fp_img", "")) if "fp_img" in d.columns else "",
            esc(r.get("fn_img", "")) if "fn_img" in d.columns else "",
            esc(r.get("err_obj", "")),
            esc(tp_ratio),
            esc(r.get("avg_confidence", "")),
            link,
        ])

    table_html = []
    table_html.append("<h3 style='margin-top:18px;'>Top images by (fp+fn) (click Open)</h3>")
    table_html.append("<div style='overflow:auto; max-height:460px; border:1px solid #ddd;'>")
    table_html.append("<table style='border-collapse:collapse; width:100%; font-family:Arial; font-size:12px;'>")
    table_html.append("<thead><tr>")
    for h in headers:
        table_html.append(
            f"<th style='position:sticky; top:0; background:#f7f7f7; border:1px solid #ddd; padding:6px; text-align:left;'>{esc(h)}</th>"
        )
    table_html.append("</tr></thead><tbody>")

    for row in rows:
        table_html.append("<tr>")
        for cell in row:
            table_html.append(f"<td style='border:1px solid #ddd; padding:6px; vertical-align:top;'>{cell}</td>")
        table_html.append("</tr>")

    table_html.append("</tbody></table></div>")
    return "\n".join(table_html)


def make_day_summary_html(df_day: pd.DataFrame, corr_gt_tp: Optional[float]) -> str:
    """
    Small summary block at the top of the day page (KISS).
    """
    def fnum(x: object, nd: int = 3) -> str:
        try:
            if pd.isna(x):
                return "-"
            return f"{float(x):.{nd}f}"
        except Exception:
            return "-"

    n = len(df_day)
    gt_total = df_day.get("gt_count_img", pd.Series([pd.NA]*n)).sum() if "gt_count_img" in df_day.columns else pd.NA
    tp_total = df_day.get("tp_img", pd.Series([pd.NA]*n)).sum() if "tp_img" in df_day.columns else pd.NA
    fp_total = df_day.get("fp_img", pd.Series([pd.NA]*n)).sum() if "fp_img" in df_day.columns else pd.NA
    fn_total = df_day.get("fn_img", pd.Series([pd.NA]*n)).sum() if "fn_img" in df_day.columns else pd.NA

    avg_gt = (gt_total / max(n, 1)) if gt_total is not pd.NA else pd.NA
    avg_tp = (tp_total / max(n, 1)) if tp_total is not pd.NA else pd.NA
    avg_fp = (fp_total / max(n, 1)) if fp_total is not pd.NA else pd.NA
    avg_fn = (fn_total / max(n, 1)) if fn_total is not pd.NA else pd.NA

    # tp_ratio summary (only where gt>0)
    if "tp_ratio" in df_day.columns:
        tp_ratio_mean = pd.to_numeric(df_day["tp_ratio"], errors="coerce").dropna().mean()
        tp_ratio_median = pd.to_numeric(df_day["tp_ratio"], errors="coerce").dropna().median()
    else:
        tp_ratio_mean = pd.NA
        tp_ratio_median = pd.NA

    corr_txt = "-" if corr_gt_tp is None or pd.isna(corr_gt_tp) else f"{float(corr_gt_tp):.3f}"

    block = []
    block.append("<div style='border:1px solid #ddd; background:#fafafa; padding:12px; margin:10px 0 18px 0; font-family:Arial;'>")
    block.append("<div style='font-size:14px; font-weight:700; margin-bottom:6px;'>Day summary</div>")
    block.append("<div style='font-size:12px; line-height:1.6;'>")
    block.append(f"Images: <b>{n}</b><br>")
    if "gt_count_img" in df_day.columns:
        block.append(f"GT total / avg per img: <b>{int(gt_total)}</b> / <b>{fnum(avg_gt)}</b><br>")
    if "tp_img" in df_day.columns:
        block.append(f"TP total / avg per img: <b>{int(tp_total)}</b> / <b>{fnum(avg_tp)}</b><br>")
    if "fp_img" in df_day.columns:
        block.append(f"FP total / avg per img: <b>{int(fp_total)}</b> / <b>{fnum(avg_fp)}</b><br>")
    if "fn_img" in df_day.columns:
        block.append(f"FN total / avg per img: <b>{int(fn_total)}</b> / <b>{fnum(avg_fn)}</b><br>")

    if "gt_count_img" in df_day.columns and "tp_img" in df_day.columns:
        block.append(f"corr(GT, TP) across images: <b>{corr_txt}</b><br>")
    if "tp_ratio" in df_day.columns:
        block.append(f"TP/GT (gt>0) mean / median: <b>{fnum(tp_ratio_mean)}</b> / <b>{fnum(tp_ratio_median)}</b><br>")

    block.append("</div></div>")
    return "\n".join(block)


# =========================
# Image-level day page: Scheme A (2 charts) + Summary + Top table
# =========================
def write_image_level_day_html(
    df_day: pd.DataFrame,
    out_html: Path,
    title: str,
    top_n: int = 20,
) -> None:
    df_day = df_day.copy()
    df_day = add_fo_url(df_day)

    # Prepare x
    x, x_title, df_day = _x_for_day(df_day)

    # Ensure numeric
    for c in ["gt_count_img", "pred_count_img", "tp_img", "fp_img", "fn_img", "avg_confidence"]:
        if c in df_day.columns:
            df_day[c] = pd.to_numeric(df_day[c], errors="coerce").fillna(0)

    # event + error
    df_day["event"] = infer_event_label(df_day)
    df_day["err_obj"] = df_day.get("fp_img", 0).fillna(0) + df_day.get("fn_img", 0).fillna(0)

    # tp_ratio (only for gt>0; else NaN)
    if "gt_count_img" in df_day.columns and "tp_img" in df_day.columns:
        gt = pd.to_numeric(df_day["gt_count_img"], errors="coerce").fillna(0)
        tp = pd.to_numeric(df_day["tp_img"], errors="coerce").fillna(0)
        df_day["tp_ratio"] = (tp / gt.replace(0, pd.NA)).astype("float")  # NaN where gt==0
    else:
        df_day["tp_ratio"] = pd.NA

    # corr(GT, TP) across images
    corr_gt_tp: Optional[float] = None
    if "gt_count_img" in df_day.columns and "tp_img" in df_day.columns:
        gts = pd.to_numeric(df_day["gt_count_img"], errors="coerce")
        tps = pd.to_numeric(df_day["tp_img"], errors="coerce")
        tmp = pd.DataFrame({"gt": gts, "tp": tps}).dropna()
        if len(tmp) >= 2:
            # Pearson correlation on counts
            corr_gt_tp = float(tmp["gt"].corr(tmp["tp"]))

    # Hover cols (shared)
    hover_cols = _make_hover_cols(df_day)

    # -------- Chart 1: GT vs TP (+ TP/GT on y2) --------
    fig1 = go.Figure()

    if "gt_count_img" in df_day.columns:
        fig1.add_trace(
            go.Scatter(
                x=x,
                y=df_day["gt_count_img"],
                mode="lines+markers",
                name="GT count (per image)",
                customdata=df_day[["url"] + hover_cols].values,
                hovertemplate=_build_hovertemplate("GT count", hover_cols),
            )
        )
    if "tp_img" in df_day.columns:
        fig1.add_trace(
            go.Scatter(
                x=x,
                y=df_day["tp_img"],
                mode="lines+markers",
                name="TP (per image)",
                customdata=df_day[["url"] + hover_cols].values,
                hovertemplate=_build_hovertemplate("TP", hover_cols),
            )
        )
    # TP/GT ratio on y2 (kept in the same figure; still KISS)
    if "tp_ratio" in df_day.columns and df_day["tp_ratio"].notna().any():
        fig1.add_trace(
            go.Scatter(
                x=x,
                y=df_day["tp_ratio"],
                mode="lines+markers",
                name="TP/GT (gt>0)",
                yaxis="y2",
                customdata=df_day[["url"] + hover_cols].values,
                hovertemplate=_build_hovertemplate("TP/GT", hover_cols),
            )
        )

    fig1.update_layout(
        title=f"{title} | A1) GT vs TP (+ TP/GT on right axis) (click points)",
        xaxis=dict(title=x_title, rangeslider=dict(visible=True)),
        yaxis=dict(title="Count"),
        yaxis2=dict(title="TP/GT", overlaying="y", side="right"),
        template="plotly_white",
        margin=dict(l=70, r=70, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    # -------- Chart 2: FP / FN / (FP+FN) --------
    fig2 = go.Figure()
    for col, nm, ylab in [
        ("fp_img", "FP (per image)", "FP"),
        ("fn_img", "FN (per image)", "FN"),
        ("err_obj", "FP+FN (per image)", "FP+FN"),
    ]:
        if col in df_day.columns:
            fig2.add_trace(
                go.Scatter(
                    x=x,
                    y=df_day[col],
                    mode="lines+markers",
                    name=nm,
                    customdata=df_day[["url"] + hover_cols].values,
                    hovertemplate=_build_hovertemplate(ylab, hover_cols),
                )
            )

    fig2.update_layout(
        title=f"{title} | A2) FP / FN / (FP+FN) (click points)",
        xaxis=dict(title=x_title, rangeslider=dict(visible=True)),
        yaxis=dict(title="Count"),
        template="plotly_white",
        margin=dict(l=70, r=70, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    # Click to open FiftyOne for BOTH figures in the same HTML doc
    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plots = document.getElementsByClassName('js-plotly-plot');
        if (!plots || !plots.length) return;

        for (var i=0; i<plots.length; i++) {
            (function(plot){
                plot.on('plotly_click', function(e) {
                    var url = e?.points?.[0]?.customdata?.[0];
                    if (url) window.open(url, '_blank');
                });
            })(plots[i]);
        }
    });
    """

    out_html.parent.mkdir(parents=True, exist_ok=True)

    summary_html = make_day_summary_html(df_day, corr_gt_tp=corr_gt_tp)

    html1 = fig1.to_html(include_plotlyjs="cdn", full_html=False)
    html2 = fig2.to_html(include_plotlyjs=False, full_html=False)  # reuse plotlyjs from fig1
    table_html = make_top_table_html(df_day, top_n=top_n)

    full = []
    full.append("<html><head><meta charset='utf-8'><title>Image-level</title></head><body>")
    full.append(summary_html)
    full.append(html1)
    full.append(html2)
    full.append(table_html)
    full.append(f"<script>{post_script}</script>")
    full.append("</body></html>")

    out_html.write_text("\n".join(full), encoding="utf-8")
    print(f"[SAVE][image-day] {out_html}")


# =========================
# Daily overview: stacked bar + 2 lines + detailed hover + top labels
# =========================
def write_daily_overview_html(
    df: pd.DataFrame,
    out_html: Path,
    day_to_file: Dict[str, str],
    title: str,
) -> None:
    daily = (
        df.groupby("capture_date", dropna=False)
          .agg(
              images=("sample_id", "count") if "sample_id" in df.columns else ("filepath", "count"),
              hit=("hit_img", "sum"),
              miss=("miss_img", "sum"),
              false_alarm=("false_alarm_img", "sum"),
              correct_reject=("correct_reject_img", "sum"),
              gt_total=("gt_count_img", "sum") if "gt_count_img" in df.columns else ("hit_img", "sum"),
              pred_total=("pred_count_img", "sum") if "pred_count_img" in df.columns else ("hit_img", "sum"),
              tp_total=("tp_img", "sum") if "tp_img" in df.columns else ("hit_img", "sum"),
              fp_total=("fp_img", "sum") if "fp_img" in df.columns else ("hit_img", "sum"),
              fn_total=("fn_img", "sum") if "fn_img" in df.columns else ("hit_img", "sum"),
          )
          .reset_index()
    ).sort_values("capture_date")

    daily["hit_rate"] = daily["hit"] / daily["images"].clip(lower=1)
    daily["error_rate"] = (daily["miss"] + daily["false_alarm"]) / daily["images"].clip(lower=1)

    denom = (daily["tp_total"] + daily["fp_total"] + daily["fn_total"]).replace(0, pd.NA)
    daily["obj_error_rate"] = ((daily["fp_total"] + daily["fn_total"]) / denom).astype("float")

    daily["avg_gt_per_img"] = (daily["gt_total"] / daily["images"].clip(lower=1)).astype("float")
    daily["avg_pred_per_img"] = (daily["pred_total"] / daily["images"].clip(lower=1)).astype("float")

    daily["drill_html"] = daily["capture_date"].map(day_to_file).fillna("")
    x = daily["capture_date"]

    fig = go.Figure()

    hover_pack = daily[[
        "drill_html",
        "images",
        "hit", "miss", "false_alarm", "correct_reject",
        "hit_rate", "error_rate",
        "gt_total", "pred_total", "tp_total", "fp_total", "fn_total",
        "avg_gt_per_img", "avg_pred_per_img",
        "obj_error_rate",
    ]].values

    def daily_hovertemplate(series_name: str) -> str:
        return (
            "Date: %{x}<br>"
            "<b>" + series_name + "</b>: %{y}<br>"
            "Images: %{customdata[1]}<br>"
            "Hit/Miss/FA/CR: %{customdata[2]}/%{customdata[3]}/%{customdata[4]}/%{customdata[5]}<br>"
            "Hit rate (img): %{customdata[6]:.3f}<br>"
            "Error rate (img): %{customdata[7]:.3f}<br>"
            "GT total: %{customdata[8]} | Pred total: %{customdata[9]}<br>"
            "TP/FP/FN: %{customdata[10]}/%{customdata[11]}/%{customdata[12]}<br>"
            "avg GT/img: %{customdata[13]:.3f} | avg Pred/img: %{customdata[14]:.3f}<br>"
            "Obj error rate (FP+FN)/(TP+FP+FN): %{customdata[15]:.3f}<br>"
            "<b>Click</b> to drill down<br>"
            "<extra></extra>"
        )

    for col, name in [
        ("hit", "Hit (GT=1 & Pred=1)"),
        ("miss", "Miss (GT=1 & Pred=0)"),
        ("false_alarm", "False Alarm (GT=0 & Pred=1)"),
        ("correct_reject", "Correct Reject (GT=0 & Pred=0)"),
    ]:
        fig.add_trace(
            go.Bar(
                x=x,
                y=daily[col],
                name=name,
                customdata=hover_pack,
                hovertemplate=daily_hovertemplate(name),
            )
        )

    fig.add_trace(
        go.Scatter(
            x=x,
            y=daily["hit_rate"],
            name="Hit rate (image)",
            mode="lines+markers",
            yaxis="y2",
            customdata=hover_pack,
            hovertemplate=daily_hovertemplate("Hit rate (image)"),
        )
    )
    fig.add_trace(
        go.Scatter(
            x=x,
            y=daily["error_rate"],
            name="Error rate (image) = (miss+FA)/images",
            mode="lines+markers",
            yaxis="y2",
            customdata=hover_pack,
            hovertemplate=daily_hovertemplate("Error rate (image)"),
        )
    )

    # top labels: show images count at top of stacked bar
    fig.add_trace(
        go.Scatter(
            x=x,
            y=daily["images"],
            mode="text",
            text=[f"{int(v)}" for v in daily["images"].fillna(0)],
            textposition="top center",
            showlegend=False,
            hoverinfo="skip",
        )
    )

    fig.update_layout(
        title=title,
        barmode="stack",
        xaxis=dict(title="capture_date", rangeslider=dict(visible=True)),
        yaxis=dict(title="Images (stacked by outcome)"),
        yaxis2=dict(title="Rate", overlaying="y", side="right", range=[0, 1]),
        template="plotly_white",
        margin=dict(l=70, r=70, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plot = document.getElementsByClassName('js-plotly-plot')[0];
        if (!plot) return;

        plot.on('plotly_click', function(e) {
            var rel = e?.points?.[0]?.customdata?.[0];
            if (rel) window.open(rel, '_blank');
        });
    });
    """

    out_html.parent.mkdir(parents=True, exist_ok=True)
    fig.write_html(out_html, include_plotlyjs="cdn", full_html=True, post_script=post_script)
    print(f"[SAVE][daily] {out_html}")


# =========================
# Main
# =========================
def process_one_csv(csv_path: Path) -> None:
    df = pd.read_csv(csv_path)

    if "filepath" not in df.columns:
        print(f"[SKIP] missing filepath: {csv_path}")
        return

    df = ensure_time_cols(df)
    df = ensure_event_cols(df)

    report_dir = OUT_ROOT / csv_path.parent.name / safe_name(csv_path.stem)
    report_dir.mkdir(parents=True, exist_ok=True)

    day_to_file: Dict[str, str] = {}

    for day, g in df.groupby("capture_date"):
        day = str(day)
        day_file = f"image_level_{day}.html"
        out_day_html = report_dir / day_file

        write_image_level_day_html(
            df_day=g,
            out_html=out_day_html,
            title=f"{csv_path.stem} | {day}",
            top_n=TOP_N,
        )
        day_to_file[day] = day_file

    out_daily_html = report_dir / "daily_overview.html"
    write_daily_overview_html(
        df=df,
        out_html=out_daily_html,
        day_to_file=day_to_file,
        title=f"{csv_path.stem} | Daily overview (click bars/lines to drill down)",
    )

    (report_dir / "INDEX.txt").write_text(
        "\n".join([
            f"CSV: {csv_path}",
            f"Daily: {out_daily_html.name}",
            f"Days: {len(day_to_file)}",
            "",
            "Daily page:",
            "- Stacked bars (Hit/Miss/FA/CR) + lines (Hit rate / Error rate)",
            "- Hover shows image-level + object-level summary",
            "- Labels show images count at the top of each bar",
            "- Click stacked bars OR lines -> open image_level_YYYY-MM-DD.html",
            "",
            "Image-level day page (Scheme A):",
            "- Summary block at the top includes corr(GT,TP) + TP/GT mean/median",
            "- A1) GT vs TP (lines+markers) + TP/GT on right axis",
            "- A2) FP/FN/(FP+FN) (lines+markers)",
            "- Range slider in both charts",
            "- Click any point -> FiftyOne",
            "- Top table sorted by fp+fn (desc), clickable Open links",
        ]),
        encoding="utf-8"
    )

    print(f"[DONE] report_dir = {report_dir}")


def main():
    csvs = find_per_image_csvs(PER_IMAGE_DIR)
    print("[INFO] Found CSVs:", len(csvs))
    if LIMIT is not None:
        csvs = csvs[:LIMIT]
        print("[INFO] Using first:", len(csvs))

    for p in csvs:
        print("\n" + "=" * 90)
        print("[CSV]", p)
        process_one_csv(p)

    print("\nAll done. Open the generated daily_overview.html files under:")
    print(OUT_ROOT)


if __name__ == "__main__":
    main()


[INFO] Found CSVs: 504

[CSV] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50.csv
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-06.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-07.html


/home/tianqi/miniconda3/envs/fif/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide



[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-08.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-15.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-16.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50/image_level_2024-06-17.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2

/home/tianqi/miniconda3/envs/fif/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide



[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image_level_2024-06-15.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image_level_2024-06-16.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image_level_2024-06-17.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50/image_level_2024-06-18.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2

...

[SAVE][daily] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c80__iou50/daily_overview.html
[DONE] report_dir = _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c80__iou50

[CSV] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c85__iou50.csv
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c85__iou50/image_level_2024-06-05.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c85__iou50/image_level_2024-06-06.html
[SAVE][daily] _eval_exports_per_

/home/tianqi/miniconda3/envs/fif/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide

/home/tianqi/miniconda3/envs/fif/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning:

invalid value encountered in divide

/home/tianqi/miniconda3/envs/fif/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide



[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c91__iou50/image_level_2024-06-06.html
[SAVE][daily] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c91__iou50/daily_overview.html
[DONE] report_dir = _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c91__iou50

[CSV] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c92__iou50.csv
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c92__iou50/image_level_2024-06-05.html
[SAVE][image-day] _eval_exports_

/home/tianqi/miniconda3/envs/fif/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide

/home/tianqi/miniconda3/envs/fif/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide



[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50/image_level_2024-06-05.html
[SAVE][image-day] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50/image_level_2024-06-06.html
[SAVE][daily] _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50/daily_overview.html
[DONE] report_dir = _eval_exports_per_images/sahi_null_v2/_prod_kiss_reports/sahi_null_v2_sw1_0605__0613_07_ok/image_level_a04_yolo11s_custom7null_cv1_ms2_0809__0823_10_ok_8__c93__iou50

All done. Open the generated daily_overview.html files under:
_eval_exports_per_images/sahi_null_v2/_prod_kiss_reports
